Here we present a quick example for computing matching diagrams and densities.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import scipy.spatial.distance as dist
import itertools

import tdqual.topological_data_quality_0 as tdqual

import os 
plots_dir = "plots/quickexample/"
os.makedirs(plots_dir, exist_ok=True)

Let us generate a point sample.

In [ ]:
RandGen = np.random.default_rng(2)
# # Generate Random Sample
X = tdqual.sampled_circle(0,2,30, RandGen)
S_indices = RandGen.choice(X.shape[0],15, replace=False)
X[S_indices]+=[1,0]
# Sort 
S_compl = np.ones(X.shape[0], dtype="bool")
S_compl[S_indices] = False
X = np.vstack((X[S_indices], X[S_compl]))
S_indices = range(len(S_indices))
S = X[S_indices]
# Plot point cloud
fig, ax = plt.subplots(ncols=1, figsize=(3,3))
ax.scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=60, marker="o", zorder=2)
ax.scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=40, marker="x", zorder=1)
ax.set_axis_off()
plt.savefig(plots_dir + "points_0.png")

Next, we compute the endpoints of the domain, the codomain and the matching.

In [ ]:
filtration_list_X, pairs_arr_X = tdqual.filtration_pairs(X) # Codomain endpoints
filtration_list_S, pairs_arr_S = tdqual.filtration_pairs(S) # Domain endpoints
F = tdqual.get_inclusion_matrix(pairs_arr_S, pairs_arr_X, S_indices) # Associated matrix
matching = tdqual.get_inclusion_matrix_pivots(F, X.shape[0]) # Matching

Let us compute the matching barcodes.

In [ ]:
fig, ax = plt.subplots(figsize=(7,2.5))
tdqual.plot_matching_0(filtration_list_S, filtration_list_X, matching, ax)
plt.tight_layout()
plt.savefig(plots_dir + "block_matching_0.png")

Next, we plot the matching diagram.

In [ ]:
fig, ax = plt.subplots(figsize=(3,3))
D_f, multiplicities = tdqual.compute_matching_diagram(filtration_list_S, filtration_list_X, matching, _tol=1e-5)
tdqual.plot_matching_diagram(D_f, ax)
plt.tight_layout()
plt.savefig(plots_dir + "matching_diagram_0.png")

And finally, we plot the density of this matching diagram. For this, we compute first a representation of $D(f)$ and read also the points from the cokernel.

In [ ]:
# Representation of D(f)
D_f_rep = []
for i, pair in enumerate(D_f):
    for j in range(multiplicities[i]):
        D_f_rep += list(pair)

D_f_rep = np.array(D_f_rep).reshape(-1,2) # saved into a numpy array
# Representation of coker(f)
coker_f_S = D_f[D_f[:,0]==np.inf][:,1]
coker_f_mult = np.array(multiplicities)[D_f[:,0]==np.inf]
coker_f_rep = []
for i, end in enumerate(coker_f_S):
    coker_f_rep += [end]*coker_f_mult[i]

In [ ]:
tdqual.plot_density_matching_diagram(D_f_rep, coker_f_rep, plots_dir + "density_matrix_0.png", nbins=5, show_colorbar=True)